In [1]:
# from sklearn import svm,metrics
# from sklearn.neighbors import KNeighborsClassifier  #분류
# from sklearn.neighbors import KNeighborsRegressor  #예측
from sklearn.linear_model import LinearRegression  #선형회귀
from sklearn.preprocessing import PolynomialFeatures  #다항특성을 만들어주는 라이브러리

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
matplotlib.rcParams['font.family']='Malgun Gothic'  #한글설정
matplotlib.rcParams['font.size']= '10'  #글자크기
matplotlib.rcParams['axes.unicode_minus'] = False

In [25]:
#타이타닉의 생존자 분류 모델을 구현하시오.
#target: survived
#data: 그외
#분류: knn, logistric, RandomForest
df_train=pd.read_csv('../z20_data/titanic_train.csv')
df_train.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0000,NaN,S,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",female,30.0,1,0,SC/PARIS 2148,13.8583,NaN,C,NaN,"Barcelona, Spain / Havana, Cuba"


In [45]:
df_test=pd.read_csv('../z20_data/titanic_test.csv')
df_test.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,3,0,"Rekic, Mr. Tido",male,38.0,0,0,349249,7.8958,NaN,S,NaN,NaN
1,2,1,"Phillips, Miss. Alice Frances Louisa",female,21.0,0,1,S.O./P.P. 2,21.0000,NaN,S,NaN,"Ilfracombe, Devon"
2,2,0,"Jacobsohn, Mr. Sidney Samuel",male,42.0,1,0,243847,27.0000,NaN,S,NaN,London


In [92]:
df_train['age'].astype(int)
df_train

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",1,13.000000,0,1,250644,19.5000,NaN,NaN,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",1,4.000000,1,1,29103,23.0000,NaN,NaN,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",1,30.000000,1,0,SC/PARIS 2148,13.8583,NaN,NaN,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,0,"Scanlan, Mr. James",0,30.231444,0,0,36209,7.7250,NaN,NaN,NaN,NaN
4,3,1,"Bradley, Miss. Bridget Delia",1,22.000000,0,0,334914,7.7250,NaN,NaN,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,3,1,"Dean, Miss. Elizabeth Gladys ""Millvina""",1,0.170000,1,2,C.A. 2315,20.5750,NaN,NaN,NaN,"Devon, England Wichita, KS"
912,3,0,"Guest, Mr. Robert",0,30.231444,0,0,376563,8.0500,NaN,NaN,NaN,NaN
913,3,1,"Smyth, Miss. Julia",1,30.231444,0,0,335432,7.7333,NaN,NaN,NaN,NaN
914,2,1,"Sincock, Miss. Maude",1,20.000000,0,0,C.A. 33112,36.7500,NaN,NaN,NaN,"Cornwall / Hancock, MI"


In [48]:
df_test.isnull().sum()

pclass         0
survived       0
name           0
sex            0
age           88
sibsp          0
parch          0
ticket         0
fare           0
cabin        312
embarked       0
body         357
home.dest    175
dtype: int64

In [36]:
df_train.shape

(916, 13)

In [38]:
print('데이터 세트 Null 값 갯수 ',df_train.isnull().sum())

데이터 세트 Null 값 갯수  pclass         0
survived       0
name           0
sex            0
age          175
sibsp          0
parch          0
ticket         0
fare           0
cabin        702
embarked       2
body         831
home.dest    389
dtype: int64


##### 데이터 피처 설명
- pclass : Passenger Class, 승객 등급
- survived : 생존 여부 : target 값이 됨.
- name : 승객 이름
- sex : 승객 성별
- age : 승객 나이
- sibsp : 탑승 한 형제/배우자 수
- parch : 탑승 한 부모/자녀 수
- ticket : 티켓 번호
- fare : 승객 지불 요금
- cabin : 선실 이름
- embarked : 승선항 (C = 쉘 부르그, Q = 퀸즈타운, S = 사우스 햄튼)
- body : 사망자 확인 번호 - 분석과정에서 제외해야 함. 머신러닝에 100% 영향을 미침.
- home.dest : 고향/목적지

In [44]:
def drop_features(df_train):
    df_train.drop(['name','cabin','body','home.dest'],axis=1,inplace=True)
    return df_train
  

In [104]:
df_train['fare'].nunique()

243

In [43]:
df_train.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'body', 'home.dest'],
      dtype='object')

In [105]:
df_train['embarked']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
911   NaN
912   NaN
913   NaN
914   NaN
915   NaN
Name: embarked, Length: 916, dtype: float64

In [109]:
df_train.head(5)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",1,13.000000,0,1,250644,19.5000,NaN,NaN,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",1,4.000000,1,1,29103,23.0000,NaN,NaN,NaN,"Cornwall / Akron, OH"
2,2,1,"Duran y More, Miss. Florentina",1,30.000000,1,0,SC/PARIS 2148,13.8583,NaN,NaN,NaN,"Barcelona, Spain / Havana, Cuba"
3,3,0,"Scanlan, Mr. James",0,30.231444,0,0,36209,7.7250,NaN,NaN,NaN,NaN
4,3,1,"Bradley, Miss. Bridget Delia",1,22.000000,0,0,334914,7.7250,NaN,NaN,NaN,"Kingwilliamstown, Co Cork, Ireland Glens Falls..."


In [113]:
pd.get_dummies(data,columns=['embarked'],drop_first = True)

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked_Q,embarked_S
0,2,1,1,13.000000,0,1,250644,19.5000,NaN,False,True
1,2,1,1,4.000000,1,1,29103,23.0000,NaN,False,True
2,2,1,1,30.000000,1,0,SC/PARIS 2148,13.8583,NaN,False,False
3,3,0,0,30.231444,0,0,36209,7.7250,NaN,True,False
4,3,1,1,22.000000,0,0,334914,7.7250,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...
911,3,1,1,0.170000,1,2,C.A. 2315,20.5750,NaN,False,True
912,3,0,0,30.231444,0,0,376563,8.0500,NaN,False,True
913,3,1,1,30.231444,0,0,335432,7.7333,NaN,True,False
914,2,1,1,20.000000,0,0,C.A. 33112,36.7500,NaN,False,True


In [ ]:
#데이터 확인:nan,0,타입,...
#데이터 전처리 - nan,0, 
#nan data 처리?
#train, test 세트
#각각 모델을 적용
#정답률 체크
#분류: knn, 로지스틱,랜덤포레스트

In [8]:
df_train['survived'].value_counts()

survived
0    563
1    353
Name: count, dtype: int64

In [15]:
df_train.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,916.000000,916.000000,741.000000,916.000000,916.000000,916.000000,85.000000
mean,2.292576,0.385371,30.231444,0.532751,0.371179,32.532050,157.870588
std,0.842576,0.486949,13.934240,1.125957,0.837508,47.416524,98.257653
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,68.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,148.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,249.000000
max,3.000000,1.000000,74.000000,8.000000,9.000000,512.329200,328.000000


In [50]:
df_train['sex']=df_train['sex'].map({'male':0, 'female':1})
df_test['sex']=df_test['sex'].map({'male':0, 'female':1})

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     916 non-null    int64  
 1   survived   916 non-null    int64  
 2   name       916 non-null    object 
 3   sex        916 non-null    object 
 4   age        741 non-null    float64
 5   sibsp      916 non-null    int64  
 6   parch      916 non-null    int64  
 7   ticket     916 non-null    object 
 8   fare       916 non-null    float64
 9   cabin      214 non-null    object 
 10  embarked   914 non-null    object 
 11  body       85 non-null     float64
 12  home.dest  527 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 93.2+ KB


In [85]:
data=df_train.drop(['name','body','home.dest'],axis=1)
data.head(3)
test=df_test.drop(['name','body','home.dest'],axis=1)
test.head(3)
x_data=data[['pclass','sex','age','sibsp','parch','fare','embarked']]
x_data
# y_data=data['survived'].to_numpy()
# y_data

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,1,13.000000,0,1,19.5000,S
1,2,1,4.000000,1,1,23.0000,S
2,2,1,30.000000,1,0,13.8583,C
3,3,0,30.231444,0,0,7.7250,Q
4,3,1,22.000000,0,0,7.7250,Q
...,...,...,...,...,...,...,...
911,3,1,0.170000,1,2,20.5750,S
912,3,0,30.231444,0,0,8.0500,S
913,3,1,30.231444,0,0,7.7333,Q
914,2,1,20.000000,0,0,36.7500,S


In [90]:
x_data['embarked']=x_data['embarked'].map({'S':1,'C':2, 'Q':3})
x_data

C:\Users\KOREAVC\AppData\Local\Temp\ipykernel_16168\3745809926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_data['embarked']=x_data['embarked'].map({'S':1,'C':2, 'Q':3})


,pclass,sex,age,sibsp,parch,fare,embarked
0,2,1,13.000000,0,1,19.5000,NaN
1,2,1,4.000000,1,1,23.0000,NaN
2,2,1,30.000000,1,0,13.8583,NaN
3,3,0,30.231444,0,0,7.7250,NaN
4,3,1,22.000000,0,0,7.7250,NaN
...,...,...,...,...,...,...,...
911,3,1,0.170000,1,2,20.5750,NaN
912,3,0,30.231444,0,0,8.0500,NaN
913,3,1,30.231444,0,0,7.7333,NaN
914,2,1,20.000000,0,0,36.7500,NaN


In [91]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(x_data)
train_scaler=ss.transform(x_data)
x_data

c:\Python312\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Python312\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Python312\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


,pclass,sex,age,sibsp,parch,fare,embarked
0,2,1,13.000000,0,1,19.5000,NaN
1,2,1,4.000000,1,1,23.0000,NaN
2,2,1,30.000000,1,0,13.8583,NaN
3,3,0,30.231444,0,0,7.7250,NaN
4,3,1,22.000000,0,0,7.7250,NaN
...,...,...,...,...,...,...,...
911,3,1,0.170000,1,2,20.5750,NaN
912,3,0,30.231444,0,0,8.0500,NaN
913,3,1,30.231444,0,0,7.7333,NaN
914,2,1,20.000000,0,0,36.7500,NaN


In [80]:
x_data

,pclass,sex,age,sibsp,parch,fare
0,2,1,13.000000,0,1,19.5000
1,2,1,4.000000,1,1,23.0000
2,2,1,30.000000,1,0,13.8583
3,3,0,30.231444,0,0,7.7250
4,3,1,22.000000,0,0,7.7250
...,...,...,...,...,...,...
911,3,1,0.170000,1,2,20.5750
912,3,0,30.231444,0,0,8.0500
913,3,1,30.231444,0,0,7.7333
914,2,1,20.000000,0,0,36.7500


In [ ]:
target=np.array(df_train['survived'])
target

In [ ]:

train_input,train_target,test_input,test_target=train_test_split(
    data,target,random_state=42
)